In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import json

# Load the dataset
df = pd.read_csv('mha_dataset.csv', parse_dates=['REPORT_DATE'])

# Preprocess the data
df['REPORT_MONTH'] = df['REPORT_DATE'].dt.month
df['REPORT_YEAR'] = df['REPORT_DATE'].dt.year

# Encode categorical variables
le = LabelEncoder()
df['PREMISES_TYPE_ENCODED'] = le.fit_transform(df['PREMISES_TYPE'])

# Prepare features and target
X = df[['REPORT_MONTH', 'REPORT_YEAR']]
y = df['PREMISES_TYPE_ENCODED']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Generate predictions for the next 12 months
last_date = df['REPORT_DATE'].max()
next_12_months = pd.date_range(start=last_date + timedelta(days=1), periods=12, freq='M')
future_data = pd.DataFrame({
    'REPORT_MONTH': next_12_months.month,
    'REPORT_YEAR': next_12_months.year
})

# Make predictions
future_predictions = rf_model.predict(future_data)

# Convert encoded predictions back to original labels
future_premises_types = le.inverse_transform(future_predictions)

# Create a dictionary with month_year as keys and premises_type as values
result_dict = {f"{date.strftime('%B_%Y')}": premises_type 
               for date, premises_type in zip(next_12_months, future_premises_types)}

# Convert the dictionary to JSON
json_output = json.dumps(result_dict, indent=2)

print("Predictions for the next 12 months:")
print(json_output)

# Optionally, save the JSON to a file
with open('premises_type_predictions.json', 'w') as f:
    json.dump(result_dict, f, indent=2)


Model Accuracy: 0.41
Predictions for the next 12 months:
{
  "April_2024": "Apartment",
  "May_2024": "Apartment",
  "June_2024": "Apartment",
  "July_2024": "Apartment",
  "August_2024": "Apartment",
  "September_2024": "Apartment",
  "October_2024": "Apartment",
  "November_2024": "Apartment",
  "December_2024": "Apartment",
  "January_2025": "Apartment",
  "February_2025": "Apartment",
  "March_2025": "Apartment"
}


/var/folders/2f/s60xqpzs6gjbtzpd5c80194r0000gn/T/ipykernel_33321/3678772692.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  next_12_months = pd.date_range(start=last_date + timedelta(days=1), periods=12, freq='M')


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from datetime import datetime, timedelta
import json

# Load the dataset
df = pd.read_csv('mha_dataset.csv', parse_dates=['REPORT_DATE'])

# Preprocess the data
df['REPORT_MONTH'] = df['REPORT_DATE'].dt.month
df['REPORT_YEAR'] = df['REPORT_DATE'].dt.year

# List of features to predict
features_to_predict = ['DIVISION', 'PREMISES_TYPE', 'APPREHENSION_TYPE', 'SEX', 'AGE_COHORT', 
                       'NEIGHBOURHOOD_158', 'Apprehension_Type_Recoded', 'Division_Recoded']

# Dictionary to store label encoders
label_encoders = {}

# Encode categorical variables
for feature in features_to_predict:
    le = LabelEncoder()
    df[f'{feature}_ENCODED'] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Prepare features
X = df[['REPORT_MONTH', 'REPORT_YEAR']]

# Dictionary to store models
models = {}

# Train a model for each feature
for feature in features_to_predict:
    y = df[f'{feature}_ENCODED']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{feature} Model Accuracy: {accuracy:.2f}")
    
    models[feature] = rf_model

# Generate predictions for the next 12 months
last_date = df['REPORT_DATE'].max()
next_12_months = pd.date_range(start=last_date + timedelta(days=1), periods=12, freq='M')
future_data = pd.DataFrame({
    'REPORT_MONTH': next_12_months.month,
    'REPORT_YEAR': next_12_months.year
})

# Make predictions for each feature
predictions = {}
for feature in features_to_predict:
    future_predictions = models[feature].predict(future_data)
    predictions[feature] = label_encoders[feature].inverse_transform(future_predictions)

# Create a dictionary with month_year as keys and all predicted features as values
result_dict = {}
for i, date in enumerate(next_12_months):
    month_year = date.strftime('%B_%Y')
    result_dict[month_year] = {feature: predictions[feature][i] for feature in features_to_predict}

# Convert the dictionary to JSON
json_output = json.dumps(result_dict, indent=2)

print("Predictions for the next 12 months:")
print(json_output)

# Optionally, save the JSON to a file
with open('multi_feature_predictions.json', 'w') as f:
    json.dump(result_dict, f, indent=2)

print("\nPredictions have been saved to 'multi_feature_predictions.json'")

# Calculate and print average predictions for each feature
print("\nAverage predictions for each feature:")
for feature in features_to_predict:
    feature_predictions = [result_dict[month][feature] for month in result_dict]
    feature_counts = pd.Series(feature_predictions).value_counts()
    print(f"\n{feature}:")
    print(feature_counts.to_string())

DIVISION Model Accuracy: 0.11
PREMISES_TYPE Model Accuracy: 0.41
APPREHENSION_TYPE Model Accuracy: 0.79
SEX Model Accuracy: 0.57
AGE_COHORT Model Accuracy: 0.28
NEIGHBOURHOOD_158 Model Accuracy: 0.03
Apprehension_Type_Recoded Model Accuracy: 0.79
Division_Recoded Model Accuracy: 0.11
Predictions for the next 12 months:
{
  "April_2024": {
    "DIVISION": "D14",
    "PREMISES_TYPE": "Apartment",
    "APPREHENSION_TYPE": "Mha Sec 17 (Power Of App)",
    "SEX": "Male",
    "AGE_COHORT": "25-34",
    "NEIGHBOURHOOD_158": "South Parkdale",
    "Apprehension_Type_Recoded": "Police Officer's Power of Apprehension",
    "Division_Recoded": "350 Dovercourt Rd"
  },
  "May_2024": {
    "DIVISION": "D14",
    "PREMISES_TYPE": "Apartment",
    "APPREHENSION_TYPE": "Mha Sec 17 (Power Of App)",
    "SEX": "Male",
    "AGE_COHORT": "25-34",
    "NEIGHBOURHOOD_158": "Annex",
    "Apprehension_Type_Recoded": "Police Officer's Power of Apprehension",
    "Division_Recoded": "350 Dovercourt Rd"
  },
  "J

/var/folders/2f/s60xqpzs6gjbtzpd5c80194r0000gn/T/ipykernel_33321/3483026615.py:52: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  next_12_months = pd.date_range(start=last_date + timedelta(days=1), periods=12, freq='M')
